<a href="https://colab.research.google.com/github/raulbenitez/CHESS_ROBOT/blob/master/Training%20Repositories/Training_Method3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set environment and networks

**Import libraries and datasets:**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2

#Input Keras and Tensorflow Environment
%tensorflow_version 1.15.0
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, utils, regularizers, applications
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

print(tf.__version__)

import os

print("\nEnviornments imported")

folder = './dataset'
!unzip -q "./drive/My Drive/dataset-v3.zip" -d "dataset"

print("\nDataset loaded")


`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.15.0`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.
1.15.2

Enviornments imported
replace dataset/validate/-/im (1).jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N

Dataset loaded


**Load CNN and remove top layers:**

In [ ]:
folder_model = 'vgg19_tfm.h5'
folder_model = 'drive/My Drive/TFM/models/vggmodel19_cam2_v1.h5'
vggmodel19 = models.load_model(folder_model)
    
episodes = 100   
vggmodel19.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
vggmodel19.summary()

vggmodel19.save('final.h5')



Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [ ]:
model = models.Sequential()
for layer in vggmodel19.layers[:-3]: # go through until last layer
    model.add(layer)

model.summary()

model.save('vggmodel19.h5')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, None, None, 256)  

**Load dataset:**

In [ ]:
print("2. Create dataset:\n")

train_folder = folder + '/train'
test_folder = folder + '/validate'

print(train_folder)
print(test_folder)

clases = ["-", "BB", "BK", "BN", "BP", "BQ", "BR", "WB", "WK", "WN", "WP", "WQ", "WR"]



2. Create dataset:

./dataset/train
./dataset/validate


#Creating and Training the SVM:

**Use CNN as feature extracture:**

In [ ]:
img_max = 775

feat_train = []
pred_train = []
true_train = []

dim = (224, 224)

for i in range(0,len(clases)):

  cl = clases[i]
  fld = train_folder + '/' + cl


  for n in range(1,img_max):  

    fl = fld +"/im (" + str(n) +").jpg"

    try: 
      img = image.load_img(fl)
      img = image.img_to_array(img)
      img = cv2.resize(img,dim)
      img = np.expand_dims(img/255., axis=0)

      pred = model.predict(img)
      feat_train.append(pred)
      pred_train.append(np.argmax(pred))
      true_train.append(i)
    
    except:
      print("error loading img")


print(np.shape(feat_train))
print("Train files predicted")

(10062, 1, 1024)
Train files predicted


In [ ]:
img_max = 175

feat_test = []
pred_test = []
true_test = []

dim = (224, 224)

for i in range(0,len(clases)):

  cl = clases[i]
  fld = train_folder + '/' + cl

  for n in range(1,img_max):  

    fl = fld +"/im (" + str(n) +").jpg"

    try: 
      img = image.load_img(fl)
      img = image.img_to_array(img)
      img = cv2.resize(img,dim)
      img = np.expand_dims(img/255., axis=0)

      pred = model.predict(img)
      predcnn = vggmodel19.predict(img)
      feat_test.append(pred.flatten())
      pred_test.append(np.argmax(predcnn))
      true_test.append(i)

    except:
      print("Image load error") 

print(np.shape(feat_test))
print("Test files predicted")

(2262, 1024)
Test files predicted


In [ ]:
feat_test = np.squeeze(feat_test)
feat_train = np.squeeze(feat_train)

true_train = np.squeeze(true_train)
pred_train = np.squeeze(pred_train)

true_test = np.squeeze(true_test)
pred_test = np.squeeze(pred_test)

print("Train: ", np.shape(true_train), np.shape(pred_train))
print("Test: ", np.shape(true_test), np.shape(pred_test))
print("Feats:", np.shape(feat_test), np.shape(feat_train))

Train:  (10062,) (10062,)
Test:  (2262,) (2262,)
Feats: (2262, 1024) (10062, 1024)


**Fit SVM:**

In [ ]:
from keras.models import Model
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix

clf1 = SVC(kernel='linear', class_weight='balanced',gamma=0.001)
ddtrain = np.squeeze(feat_train)
clf1 = clf1.fit(ddtrain, true_train)

print(clf1)

Using TensorFlow backend.


SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


**Test SVM:**

In [ ]:
ddtest = feat_test
pred_svm = clf1.predict(ddtest)

print("SVM Results: ")
print(classification_report(true_test, pred_svm))
print(confusion_matrix(true_test, pred_svm))

print("\n\nCNN Results: ")
print(classification_report(true_test, pred_test))
print(confusion_matrix(true_test, pred_test))


SVM Results: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       174
           1       1.00      0.99      1.00       174
           2       0.99      1.00      1.00       174
           3       0.99      0.99      0.99       174
           4       1.00      0.99      0.99       174
           5       1.00      1.00      1.00       174
           6       0.99      1.00      0.99       174
           7       1.00      1.00      1.00       174
           8       1.00      1.00      1.00       174
           9       1.00      1.00      1.00       174
          10       1.00      1.00      1.00       174
          11       1.00      1.00      1.00       174
          12       1.00      1.00      1.00       174

    accuracy                           1.00      2262
   macro avg       1.00      1.00      1.00      2262
weighted avg       1.00      1.00      1.00      2262

[[174   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0 173 

**Save SVM model:**

In [ ]:

from sklearn.externals import joblib

# Save to file in the current working directory
joblib_file = "drive/My Drive/TFM/models/svm_tfm.pkl"
joblib_file = "svm_tfm_final.pkl"
joblib.dump(clf1, joblib_file)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['svm_tfm_final.pkl']

**Optimizing with Grid Search:**

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svm = SVC()
grid_search = GridSearchCV(svm, parameters)
grid_search.fit(ddtrain, true_train)
grid_search.best_estimator_

print(grid_search)

GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)


**Save model from Grid Search:**

In [ ]:
ddtest = feat_test
pred_svm = clf1.predict(ddtest)
pred_gs = grid_search.predict(ddtest)

print("SVM with Grid Search predicted")

# Save to file in the current working directory
joblib_file = "drive/My Drive/TFM/models/gridsearch_tfm.pkl"
joblib.dump(grid_search, joblib_file)


#Testing and comparing results

**Generating confusion matrices:**

In [ ]:
success = 0
success_svm = 0
success_gs = 0
success_top3 = 0
total = 0
ang = 0
img_max = 175
dim = (224, 224)

correct_answer = []
cnn_result = []
svm_result = []
gs_result = []

for lbl in clases:

    for n in range(1,img_max+1):

            file = test_folder +'/'+ lbl + "/im (" + str(n) +").jpg"

            img = image.load_img(file)
            img = image.img_to_array(img)
            img = cv2.resize(img,dim)
            img = np.expand_dims(img/255., axis=0)

            pred = vggmodel19.predict(img, batch_size = 1)
            feat = model.predict(img, batch_size = 1)

            ind = np.argsort(pred)[0][-3:]

            top1 = clases[ind[2]]
            top3 = [clases[ind[0]], clases[ind[1]], clases[ind[2]]]

            if lbl == top1:
                success += 1
            if lbl in top3:
                success_top3 += 1

            dd = np.expand_dims(np.squeeze(feat.flatten()), axis=0)
            pred_svm1 = clf1.predict(dd)
            pred_gs = grid_search.predict(dd)

            top1_svm = clases[pred_svm1[0]]
            top1_gs = clases[pred_gs[0]]

            if lbl == top1_svm:
                success_svm += 1
            if lbl == top1_gs:
              success_gs += 1

            total +=1

            correct_answer.append(lbl)
            cnn_result.append(top1)
            svm_result.append(top1_svm)
            gs_result.append(top1_gs)


print("\nSuccess rate: ", 100*success/total, "% of ", total, " images." )
print("Success rate with SVM: ", 100*success_svm/total, "% of ", total, " images." )
print("Success rate with Grid Search: ", 100*success_gs/total, "% of ", total, " images." )
print("Success rate of top 3: ", 100*success_top3/total, "% of ", total, " images.\n" )




In [ ]:
cf_matrix_svm = confusion_matrix(correct_answer, svm_result)
cf_matrix_gs = confusion_matrix(correct_answer, gs_result)
cf_matrix_cnn = confusion_matrix(correct_answer, cnn_result)

print("CNN Results: ")
print(cf_matrix_cnn)
print("\nSVM Results: ")
print(cf_matrix_svm)
print("\nGrid Search Results: ")
print(cf_matrix_gs)

**Saving Results:** 

In [ ]:
import pandas as pd
import seaborn as sns

df_cm_svm = pd.DataFrame(cf_matrix_svm, columns=np.unique(clases), index = np.unique(clases),)
df_cm_svm.index.name = 'Actual'
df_cm_svm.columns.name = 'Predicted SVM'

df_cm_gs = pd.DataFrame(cf_matrix_gs, columns=np.unique(clases), index = np.unique(clases))
df_cm_gs.index.name = 'Actual'
df_cm_gs.columns.name = 'Predicted GS'

df_cm_cnn = pd.DataFrame(cf_matrix_cnn, columns=np.unique(clases), index = np.unique(clases))
df_cm_cnn.index.name = 'Actual'
df_cm_cnn.columns.name = 'Predicted CNN'

columns = {}
columns['Actual'] = true_test
columns['Predicted SVM'] = svm_result
columns['Predicted CNN'] = cnn_result

fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(df_cm_svm, cmap="Blues", annot=True,annot_kws={"size": 16}, fmt='g',ax=ax)# font size
plt.figure()
plt.show()

fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(df_cm_gs, cmap="Blues", annot=True,annot_kws={"size": 16}, fmt='g',ax=ax)# font size
plt.figure()
plt.show()

fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(df_cm_cnn, cmap="Blues", annot=True,annot_kws={"size": 16}, fmt='g',ax=ax)# font size
plt.figure()
plt.show()